In [1]:
import pandas as pd

# Genetic Variant Classification

## Loading the data

In [2]:
data = pd.read_csv('data/clinvar_conflicting.csv')

/tmp/ipykernel_17271/4112643682.py:1: DtypeWarning: Columns (0,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/clinvar_conflicting.csv')


In [3]:
data.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,...,SIFT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,1168180,G,C,0.0771,0.10020,0.1066,MedGen:CN169374,NaN,not_specified,...,tolerated,benign,NaN,NaN,NaN,NaN,NaN,1.053,-0.208682,2.0
1,1,1470752,G,A,0.0000,0.00000,0.0000,"MedGen:C1843891,OMIM:607454,Orphanet:ORPHA9877...",NaN,Spinocerebellar_ataxia_21|not_provided,...,deleterious_low_confidence,benign,NaN,NaN,NaN,NaN,NaN,31.000,6.517838,-3.0
2,1,1737942,A,G,0.0000,0.00001,0.0000,"Human_Phenotype_Ontology:HP:0000486,MedGen:C00...",NaN,Strabismus|Nystagmus|Hypothyroidism|Intellectu...,...,deleterious,probably_damaging,NaN,NaN,NaN,NaN,NaN,28.100,6.061752,-1.0
3,1,2160305,G,A,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002...",NaN,Shprintzen-Goldberg_syndrome|not_provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.500,3.114491,NaN
4,1,2160305,G,T,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002",NaN,Shprintzen-Goldberg_syndrome,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.700,4.766224,-3.0


## Cleaning the data

There are columns that can be dropped as they are not related to the problem at hand

In [4]:
data = data.drop(columns = ['BAM_EDIT', 'EXON', 'CLNDISDB', 'CLNHGVS', 
         'MC', 'CLNVI', 'SYMBOL', 'Feature', 'Feature_type', 
         'BIOTYPE', 'INTRON', 'CADD_RAW'])

#### Exploring nan values

In [5]:
missing_perc = pd.DataFrame(data.isna().mean(), columns=['missing_percentage'])
missing_perc=missing_perc.sort_values(by='missing_percentage', ascending=False)
missing_perc

,missing_percentage
MOTIF_SCORE_CHANGE,0.999969
HIGH_INF_POS,0.999969
MOTIF_POS,0.999969
MOTIF_NAME,0.999969
DISTANCE,0.998343
SSR,0.998006
CLNSIGINCL,0.997438
CLNDISDBINCL,0.997438
CLNDNINCL,0.997438
PolyPhen,0.619623


We can drop the columns with over 99 percent missing data

In [6]:
to_drop = missing_perc.index[:9]
to_drop

Index(['MOTIF_SCORE_CHANGE', 'HIGH_INF_POS', 'MOTIF_POS', 'MOTIF_NAME',
       'DISTANCE', 'SSR', 'CLNSIGINCL', 'CLNDISDBINCL', 'CLNDNINCL'],
      dtype='object')

In [7]:
data = data.drop(columns=to_drop)

In [8]:
data

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDN,CLNVC,ORIGIN,...,CDS_position,Protein_position,Amino_acids,Codons,STRAND,SIFT,PolyPhen,LoFtool,CADD_PHRED,BLOSUM62
0,1,1168180,G,C,0.0771,0.10020,0.1066,not_specified,single_nucleotide_variant,1,...,522,174,E/D,gaG/gaC,1.0,tolerated,benign,NaN,1.053,2.0
1,1,1470752,G,A,0.0000,0.00000,0.0000,Spinocerebellar_ataxia_21|not_provided,single_nucleotide_variant,1,...,509,170,P/L,cCg/cTg,-1.0,deleterious_low_confidence,benign,NaN,31.000,-3.0
2,1,1737942,A,G,0.0000,0.00001,0.0000,Strabismus|Nystagmus|Hypothyroidism|Intellectu...,single_nucleotide_variant,35,...,239,80,I/T,aTc/aCc,-1.0,deleterious,probably_damaging,NaN,28.100,-1.0
3,1,2160305,G,A,0.0000,0.00000,0.0000,Shprintzen-Goldberg_syndrome|not_provided,single_nucleotide_variant,33,...,100,34,G/S,Ggc/Agc,1.0,NaN,NaN,NaN,22.500,NaN
4,1,2160305,G,T,0.0000,0.00000,0.0000,Shprintzen-Goldberg_syndrome,single_nucleotide_variant,33,...,100,34,G/C,Ggc/Tgc,1.0,NaN,NaN,NaN,24.700,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65183,X,154158201,T,G,0.0801,0.13923,0.1605,Hereditary_factor_VIII_deficiency_disease|not_...,single_nucleotide_variant,1,...,3864,1288,S,tcA/tcC,-1.0,NaN,NaN,0.00158,0.105,NaN
65184,X,154159118,C,T,0.0020,0.00060,0.0013,"not_specified|Hemophilia_A,_FVIII_Deficiency",single_nucleotide_variant,1,...,2947,983,V/I,Gta/Ata,-1.0,tolerated,benign,0.00158,0.002,3.0
65185,X,154194886,C,T,0.0125,0.00370,0.0111,"not_specified|Hemophilia_A,_FVIII_Deficiency",single_nucleotide_variant,1,...,1086,362,A,gcG/gcA,-1.0,NaN,NaN,0.00158,12.850,NaN
65186,X,154490187,T,C,0.0003,0.00034,0.0000,Non-syndromic_X-linked_intellectual_disability...,single_nucleotide_variant,1,...,543,181,T,acA/acG,-1.0,NaN,NaN,NaN,0.130,NaN


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65188 entries, 0 to 65187
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CHROM             65188 non-null  object 
 1   POS               65188 non-null  int64  
 2   REF               65188 non-null  object 
 3   ALT               65188 non-null  object 
 4   AF_ESP            65188 non-null  float64
 5   AF_EXAC           65188 non-null  float64
 6   AF_TGP            65188 non-null  float64
 7   CLNDN             65188 non-null  object 
 8   CLNVC             65188 non-null  object 
 9   ORIGIN            65188 non-null  int64  
 10  CLASS             65188 non-null  int64  
 11  Allele            65188 non-null  object 
 12  Consequence       65188 non-null  object 
 13  IMPACT            65188 non-null  object 
 14  cDNA_position     56304 non-null  object 
 15  CDS_position      55233 non-null  object 
 16  Protein_position  55233 non-null  object

As the order of the rows is determined by first the chromosome that the variant is on and then the position of the allele, we will fill the missing values in the below rows with forward fill, as the best logical guess we can make is that the closer the position, the more related the values.

For the other numerical values, we can interpolate.

In [11]:
data.select_dtypes(include=['object'])

,CHROM,REF,ALT,CLNDN,CLNVC,Allele,Consequence,IMPACT,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,SIFT,PolyPhen
0,1,G,C,not_specified,single_nucleotide_variant,C,missense_variant,MODERATE,552,522,174,E/D,gaG/gaC,tolerated,benign
1,1,G,A,Spinocerebellar_ataxia_21|not_provided,single_nucleotide_variant,A,missense_variant,MODERATE,523,509,170,P/L,cCg/cTg,deleterious_low_confidence,benign
2,1,A,G,Strabismus|Nystagmus|Hypothyroidism|Intellectu...,single_nucleotide_variant,G,missense_variant,MODERATE,632,239,80,I/T,aTc/aCc,deleterious,probably_damaging
3,1,G,A,Shprintzen-Goldberg_syndrome|not_provided,single_nucleotide_variant,A,missense_variant,MODERATE,132,100,34,G/S,Ggc/Agc,NaN,NaN
4,1,G,T,Shprintzen-Goldberg_syndrome,single_nucleotide_variant,T,missense_variant,MODERATE,132,100,34,G/C,Ggc/Tgc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65183,X,T,G,Hereditary_factor_VIII_deficiency_disease|not_...,single_nucleotide_variant,G,synonymous_variant,LOW,4035,3864,1288,S,tcA/tcC,NaN,NaN
65184,X,C,T,"not_specified|Hemophilia_A,_FVIII_Deficiency",single_nucleotide_variant,T,missense_variant,MODERATE,3118,2947,983,V/I,Gta/Ata,tolerated,benign
65185,X,C,T,"not_specified|Hemophilia_A,_FVIII_Deficiency",single_nucleotide_variant,T,synonymous_variant,LOW,1257,1086,362,A,gcG/gcA,NaN,NaN
65186,X,T,C,Non-syndromic_X-linked_intellectual_disability...,single_nucleotide_variant,C,synonymous_variant,LOW,822,543,181,T,acA/acG,NaN,NaN


data['CHROM'] must be cleaned as there are ints and strings mixed

In [17]:
data['CHROM'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X',
       11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 3, 'MT'],
      dtype=object)

In [36]:
chrom_map = {2: '2', 3: '3', 11: '11', 12: '12',13: '13',14: '14', 15: '15',16: '16', 17: '17', 18: '18', 19: '19', 
             20: '20', 21: '21', 22: '22'}

In [39]:
data['CHROM'] = data['CHROM'].replace(chrom_map)

In [40]:
data['CHROM'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X',
       'MT'], dtype=object)

In [ ]:
for column in data.columns:
    if column == 'BLOSUM62':
        data[column] = data[column].fillna(method='ffill')
    elif data[column].dtype == 'object':
        data[column] = data[column].fillna(method='ffill')
    elif data[column].dtype == 'float64':
        data[column] = data[column].interpolate()

data['LoFtool'] = data['LoFtool'].fillna(0)

data.info()

In [ ]:
data.select_dtypes(include=['object']).nunique()

In [ ]:
to_be_one_hot = ['CLNVC', 'IMPACT', 'SIFT']
to_be_mapped = ['SIFT', 'PolyPhen']
to_be_encoded = ['REF', 'ALT', 'CLNDN', 'Allele', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons']